In [1]:
pip install numpy pandas matplotlib seaborn tensorflow keras pydot graphviz

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics, optimizers
from tensorflow.keras.callbacks import Callback

import pydot
import graphviz
from keras.utils import plot_model
from keras_tqdm import TQDMNotebookCallback
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot

In [4]:
#print keras and tensorflow versions for compatability

print(keras.__version__)
print(tf.__version__)

3.5.0
2.17.0


In [5]:
#sets the plotting backend for matplotlib to be inline
#inline: meaning it will be displayed directly into the notebook instead of other window

%matplotlib inline

In [9]:
#uses certain style sheet which is 'seaborn-v0_8-notebook'

plt.style.use('seaborn-v0_8-notebook')

In [10]:
#seaborn styling

sns.set(style="white", color_codes=True)
sns.set_context("paper", rc={"font.size":14,"axes.titlesize":15,"axes.labelsize":20,
                             'xtick.labelsize':14, 'ytick.labelsize':14})

In [11]:
# Function for reparameterization trick to make model differentiable
def sampling(args):
    
    import tensorflow as tf
    # Function with args required for Keras Lambda function
    z_mean, z_log_var = args

    # Draw epsilon of the same shape from a standard normal distribution
    epsilon = K.random_normal(shape=tf.shape(z_mean), mean=0.,
                              stddev=epsilon_std)
    
    # The latent vector is non-deterministic and differentiable
    # in respect to z_mean and z_log_var
    z = z_mean + K.exp(z_log_var / 2) * epsilon
    return z


    """we need to explain mathematically what is the signifiucance of reparametriation tric
    """

In [12]:
class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    This function is borrowed from:
    https://github.com/fchollet/keras/blob/master/examples/variational_autoencoder.py
    """
    def __init__(self, **kwargs):
        # https://keras.io/layers/writing-your-own-keras-layers/
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = original_dim * metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + z_log_var_encoded - K.square(z_mean_encoded) - 
                                K.exp(z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x
    
        """explain maths behind
        """

In [13]:
#warmup

class WarmUpCallback(Callback):
    def __init__(self, beta, kappa):
        self.beta = beta
        self.kappa = kappa
    # Behavior on each epoch
    def on_epoch_end(self, epoch, logs={}):
        if K.get_value(self.beta) <= 1:
            K.set_value(self.beta, K.get_value(self.beta) + self.kappa)
            
        """explain math relevance
        """

In [14]:
np.random.seed(123)

In [32]:
#methyl_file = os.path.join('data/raw', 'BreastCancerMethylation_top100kMAD_cpg.csv')
methyl_df = pd.read_csv('~/Desktop/BreastCancerMethylation_top100kMAD_cpg.csv', index_col = 0, header  = 0)

methyl_file2 = os.path.join('data', 'Illumina-Human-Methylation-450kilmn12-hg19.annotated.csv')
methyl_df2 = pd.read_csv(methyl_file2, index_col = 0, header  = 0)

methyl_file3 = os.path.join('data', 'Full_data_covs.csv')
methyl_df3 = pd.read_csv(methyl_file3, index_col = 0, header  = 0)


/var/folders/fq/09kvyhs13yv_g91hj_p6y7m00000gn/T/ipykernel_71646/1508721002.py:5: DtypeWarning: Columns (4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  methyl_df2 = pd.read_csv(methyl_file2, index_col = 0, header  = 0)


In [33]:
print(methyl_df.shape)
methyl_df.head(2)

(1230, 100000)


,cg01682111,cg18391209,cg27260772,cg27363327,cg17882374,cg16620382,cg23357130,cg05528899,cg01893212,cg13885357,...,cg26340740,cg07552322,cg26966083,cg03222009,cg13058453,cg17525418,cg17584014,cg01617071,cg00678668,cg01986630
CpG,,,,,,,,,,,,,,,,,,,,,
7796806090_R04C01,0.148795,0.827165,0.07093,0.068544,0.951175,0.055168,0.044086,0.203048,0.566052,0.566779,...,0.752437,0.240496,0.701855,0.717532,0.603018,0.718762,0.689224,0.514234,0.649068,0.473050
6285633051_R04C01,0.819280,0.238459,0.17315,0.074701,0.443478,0.137893,0.612504,0.952400,0.626088,0.387909,...,0.797816,0.142233,0.684140,0.785218,0.522061,0.925992,0.657774,0.577316,0.718510,0.539516


In [21]:
print(methyl_df2.shape)
methyl_df2.head(2)

(485512, 31)


,AddressA,AddressB,ProbeSeqA,ProbeSeqB,Type,NextBase,Color,chr,pos,strand,...,HMEC_H3K4me3,HMEC_H3K9ac,HMEC_H3K9me3,HMEC_H3K27ac,HMEC_H3K27me3,HMEC_H3K36me3,HMEC_H3K79me2,HMEC_H3K20me1,HMEC_SuperEnhancer,MCF7_SuperEnhancer
Name,,,,,,,,,,,,,,,,,,,,,
cg00050873,32735311,31717405.0,ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAAC...,ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAAC...,I,A,Red,chrY,9363356,-,...,0,0,0,0,0,0,0,0,0,0
cg00212031,29674443,38703326.0,CCCAATTAACCACAAAAACTAAACAAATTATACAATCAAAAAAACA...,CCCAATTAACCGCAAAAACTAAACAAATTATACGATCGAAAAAACG...,I,T,Red,chrY,21239348,-,...,0,0,0,0,0,0,0,0,0,0


In [22]:
print(methyl_df3.shape)
methyl_df3.head(2)

(1332, 17)


,PAM50,SampleType,ER,PR,HER2,Gender,PathStage,HistType,Age,Study,PercentTumorCells,PercentStromalCells,PercentNormalCells,Survival,Array,Slide,ID
Basename,,,,,,,,,,,,,,,,,
7796806090_R04C01,NaN,Primary Tumor,NaN,NaN,NaN,Female,Stage I,Infiltrating Ductal Carcinoma,46.0,TCGA_BRCA,40.0,52.0,5.0,1165.0,R04C01,7.796806e+09,TCGA-A2-A3XZ-01
6285633051_R04C01,NaN,Primary Tumor,NaN,NaN,NaN,Female,Stage I,Infiltrating Ductal Carcinoma,47.0,TCGA_BRCA,NaN,NaN,NaN,1732.0,R04C01,6.285633e+09,TCGA-BH-A0B6-01


In [23]:
# Split 10% test set randomly
test_set_percent = 0.1
methyl_test_df = methyl_df.sample(frac=test_set_percent)
methyl_train_df = methyl_df.drop(methyl_test_df.index)

In [26]:
#initialize variables and hyperparameters

# Set hyper parameters
original_dim = methyl_df.shape[1]
latent_dim = 100

batch_size = 50
epochs = 50
learning_rate = 0.0005

epsilon_std = 1.0
beta = K.variable(0)
kappa = 1

In [27]:
# Input place holder for methylation data with specific input size
methyl_input = Input(shape=(original_dim, ))

# Input layer is compressed into a mean and log variance vector of size `latent_dim`
# Each layer is initialized with glorot uniform weights and each step (dense connections,
# batch norm, and relu activation) are funneled separately
# Each vector of length `latent_dim` are connected to the rnaseq input tensor
z_mean_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(methyl_input)
z_mean_dense_batchnorm = BatchNormalization()(z_mean_dense_linear)
z_mean_encoded = Activation('relu')(z_mean_dense_batchnorm)

z_log_var_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(methyl_input)
z_log_var_dense_batchnorm = BatchNormalization()(z_log_var_dense_linear)
z_log_var_encoded = Activation('relu')(z_log_var_dense_batchnorm)

# return the encoded and randomly sampled z vector
# Takes two keras layers as input to the custom sampling function layer with a `latent_dim` output
z = Lambda(sampling, output_shape=(latent_dim, ))([z_mean_encoded, z_log_var_encoded])

In [28]:
# The decoding layer is much simpler with a single layer and sigmoid activation
decoder_to_reconstruct = Dense(original_dim, kernel_initializer='glorot_uniform', activation='sigmoid')
methyl_reconstruct = decoder_to_reconstruct(z)

In [31]:
adam = optimizers.Adam(learning_rate=0.0005)
vae_layer = CustomVariationalLayer()([methyl_input, methyl_reconstruct])
#vae_layer = CustomVariationalLayer(beta=beta, original_dim=original_dim)([methyl_input, methyl_reconstruct])
vae = Model(methyl_input, vae_layer)
vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

vae.summary()

ValueError: Unrecognized keyword arguments passed to CustomVariationalLayer: {'beta': <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>, 'original_dim': 100000}